# Weeks 10-11: Renewal Processes

**Objective:** Understand renewal processes as a generalization of the Poisson process, learn the key definitions, and simulate a process to verify its long-run average behavior.

## Step 1: Build Intuition

Remember the Poisson process, which modeled events like calls arriving at a call center? A key assumption was that the time between calls followed an **Exponential distribution**. This implies the process is "memoryless."

But what if the time between events isn't memoryless? Think of replacing a machine part. The time until it fails (and is "renewed") might not be exponential. A new part is less likely to fail than an old one. Or perhaps failures tend to happen around a specific average lifetime, but with some variability.

A **renewal process** is a counting process that handles this. It's like a Poisson process, but the time between events (the "lifetimes") can follow **any** probability distribution, as long as they are independent and identically distributed (IID).

## Step 2: Understand the Core Idea

A renewal process, \(N(t)\), counts the number of "renewals" or events up to time \(t\). It is defined by one simple thing:

1.  **The Inter-arrival Times:** Let \(T_1, T_2, T_3, ...\) be the times between consecutive events. In a renewal process, these \(T_i\) are assumed to be **independent and identically distributed (IID)** positive random variables.

That's it! The distribution of these \(T_i\) can be Exponential (which makes it a Poisson process), Gamma, Weibull, Uniform, or any other valid distribution for positive values.

- **Arrival Times:** The time of the \(n\)-th event is the sum of the first \(n\) inter-arrival times: \(S_n = T_1 + T_2 + ... + T_n\).
- **Counting Process:** The number of events by time \(t\) is \(N(t) = \max\{n: S_n \le t\}\).

## Step 3: Learn the Definitions and Formulas

**Definition: Renewal Process**
A counting process \({N(t), t \ge 0}\) is a renewal process if the inter-arrival times \(T_1, T_2, ...\) are independent and identically distributed positive random variables.

--- 

**Definition: Renewal Function**
The expected number of renewals by time \(t\) is called the renewal function, \(m(t)\).
$$ m(t) = E[N(t)] $$
Unlike the Poisson process where \(E[N(t)] = \lambda t\), the renewal function for a general renewal process is often difficult to calculate exactly.

--- 

**Key Result: Elementary Renewal Theorem**
This theorem describes the long-run average rate of renewals. Let \(\mu = E[T_i]\) be the mean inter-arrival time. Then, as time goes to infinity, the average number of renewals per unit of time approaches \(1/\mu\).
$$ \lim_{t \to \infty} \frac{E[N(t)]}{t} = \frac{1}{\mu} $$
This is highly intuitive: if a light bulb lasts 1000 hours on average (\(\mu=1000\)), then in the long run, you'd expect to replace it at a rate of 1/1000 bulbs per hour.

## Step 4: Apply and Practice

Let's simulate a renewal process where the inter-arrival times are **not** exponential. A **Gamma distribution** is a good choice because it's flexible and always positive. The Exponential distribution is just a special case of the Gamma distribution.

**Scenario:** We'll model the failure of a component. The time to failure (inter-arrival time) follows a Gamma distribution with a known shape and scale. We want to simulate the process and verify the Elementary Renewal Theorem.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-whitegrid')

### Part A: Simulating a Renewal Process Path

The simulation logic is identical to the Poisson process, but we just swap the random number generator.

In [ ]:
def simulate_renewal_process(dist_params, time_period, num_steps=None):
    """
    Simulates a single path of a renewal process with Gamma-distributed inter-arrival times.
    
    Args:
        dist_params (dict): Dictionary with 'shape' and 'scale' for the Gamma distribution.
        time_period (float): The total time to simulate for.
        num_steps (int, optional): If provided, returns the process state at these steps.
        
    Returns:
        tuple: (event_times, event_counts) or (time_axis, path_at_steps).
    """
    event_times = [0]
    current_time = 0
    
    while current_time < time_period:
        # Generate the next inter-arrival time from a Gamma distribution
        inter_arrival_time = np.random.gamma(shape=dist_params['shape'], scale=dist_params['scale'])
        current_time += inter_arrival_time
        
        if current_time < time_period:
            event_times.append(current_time)
            
    event_counts = np.arange(len(event_times))
    
    if num_steps is None:
        return event_times, event_counts
    else:
        # If we need the process value at specific times for averaging
        time_axis = np.linspace(0, time_period, num_steps)
        path_at_steps = np.zeros(num_steps)
        for i, t in enumerate(time_axis):
            path_at_steps[i] = np.sum(np.array(event_times) <= t) - 1
        return time_axis, path_at_steps

# --- Simulation Parameters ---
# Gamma distribution for inter-arrival times
GAMMA_SHAPE = 2.0
GAMMA_SCALE = 1.5
T_PERIOD = 100

# The mean of a Gamma(shape, scale) is shape * scale
MU = GAMMA_SHAPE * GAMMA_SCALE

print(f"Simulating with Gamma inter-arrival times (shape={GAMMA_SHAPE}, scale={GAMMA_SCALE})")
print(f"Theoretical mean inter-arrival time (mu): {MU:.2f}")

# Run one simulation
times, counts = simulate_renewal_process({'shape': GAMMA_SHAPE, 'scale': GAMMA_SCALE}, T_PERIOD)

# Plot the renewal process path
plt.figure(figsize=(12, 6))
plt.step(times, counts, where='post')
plt.title('A Single Realization of a Renewal Process (Gamma Inter-arrivals)')
plt.xlabel('Time')
plt.ylabel('Number of Renewals N(t)')
plt.xlim(0, T_PERIOD)
plt.grid(True)
plt.show()

### Part B: Verifying the Elementary Renewal Theorem

The theory says that \(E[N(t)]/t\) should approach \(1/\mu\) as \(t\) gets large. Let's test this by running many simulations and plotting the average of \(N(t)/t\).

In [ ]:
N_SIMS = 2000
N_STEPS = 200 # Number of points to evaluate the process at
all_paths = np.zeros((N_SIMS, N_STEPS))

for i in range(N_SIMS):
    # We need the process evaluated at the same time points for each simulation
    t_axis, path = simulate_renewal_process(
        {'shape': GAMMA_SHAPE, 'scale': GAMMA_SCALE}, 
        T_PERIOD, 
        num_steps=N_STEPS
    )
    all_paths[i, :] = path

# Calculate the average N(t) across all simulations
m_t = np.mean(all_paths, axis=0)

# Calculate m(t) / t
# We avoid division by zero at t=0 by starting from the second element
avg_rate = m_t[1:] / t_axis[1:]

# Plot the result
plt.figure(figsize=(12, 6))
plt.plot(t_axis[1:], avg_rate, label='Simulated E[N(t)] / t')
plt.axhline(1/MU, color='red', linestyle='--', label=f'Theoretical Limit (1/mu = {1/MU:.4f})')
plt.title('Verifying the Elementary Renewal Theorem')
plt.xlabel('Time (t)')
plt.ylabel('Average Rate of Renewals')
plt.legend()
plt.ylim(bottom=0)
plt.show()

**Interpretation:**

The plot clearly shows that the simulated average rate of renewals, \(E[N(t)]/t\), converges to the theoretical long-run rate, \(1/\mu\), as \(t\) increases. This provides strong empirical evidence for the Elementary Renewal Theorem.

## Summary & Next Steps

In this notebook, we've generalized the Poisson process to the more flexible Renewal Process:
1.  A **renewal process** is a counting process where the times between events are IID random variables from *any* positive distribution.
2.  The Poisson process is a special case where the inter-arrival distribution is Exponential.
3.  The **Elementary Renewal Theorem** provides a powerful result about the long-run average rate of events, stating it converges to the reciprocal of the mean inter-arrival time (\(1/\mu\)).

Renewal theory is the foundation for analyzing more complex systems, especially in reliability and queuing theory.

In **Weeks 12-13**, we will apply these ideas to study **Queuing Models**, which are essential for analyzing waiting lines in systems like call centers, banks, and computer networks.